In [3]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [19]:
# importing important libraries
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras_tuner import RandomSearch

In [5]:
x = pd.read_csv("/gdrive/MyDrive/Covid Prediction AI Project/X.csv")
y = pd.read_csv("/gdrive/MyDrive/Covid Prediction AI Project/Y.csv")

In [6]:
x.head()

,Unnamed: 0,cough,fever,sore_throat,shortness_of_breath,head_ache,Female,Male,Age above 60,Abroad,Contact with confirmed
0,0,0,0,0,0,0,1,0,0,0,0
1,1,1,0,0,0,0,1,0,0,0,0
2,2,0,1,0,0,0,0,1,0,0,0
3,3,1,0,0,0,0,1,0,0,0,0
4,4,1,0,0,0,0,0,1,0,0,0


In [7]:
y.head()

,Unnamed: 0,Result
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [8]:
x.drop(columns=['Unnamed: 0'], inplace=True)
y.drop(columns=['Unnamed: 0'], inplace=True)

## Splitting the Dataset into Training, Validation and Testing Dataset.

In [9]:
covid_df = pd.concat([x,y],axis=1)
covid_df

,cough,fever,sore_throat,shortness_of_breath,head_ache,Female,Male,Age above 60,Abroad,Contact with confirmed,Result
0,0,0,0,0,0,1,0,0,0,0,0
1,1,0,0,0,0,1,0,0,0,0,0
2,0,1,0,0,0,0,1,0,0,0,0
3,1,0,0,0,0,1,0,0,0,0,0
4,1,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
520011,1,0,0,0,0,0,1,0,0,1,1
520012,1,0,0,0,1,1,0,0,0,0,1
520013,0,0,0,0,0,0,1,0,0,0,1
520014,0,1,0,0,0,0,1,0,0,0,1


In [10]:
train_df,valid_df = train_test_split(covid_df,test_size=0.20,random_state=1)
train_df,test_df = train_test_split(train_df,test_size=0.10,random_state=1)

In [11]:
print(train_df.shape)
print(valid_df.shape)
print(test_df.shape)

(374410, 11)
(104004, 11)
(41602, 11)


In [12]:
train_df = train_df.reset_index()
valid_df = valid_df.reset_index()
test_df = test_df.reset_index()

In [13]:
train_df.drop(columns=['index'],inplace=True)
valid_df.drop(columns=['index'],inplace=True)
test_df.drop(columns=['index'],inplace=True)

In [14]:
independent_cols = ['cough', 'fever', 'sore_throat', 'shortness_of_breath', 'head_ache',
                    'Female', 'Male', 'Age above 60', 'Abroad', 'Contact with confirmed']

target_col = ['Result']


In [15]:
x_train = train_df[independent_cols].copy()
y_train = train_df[target_col].copy()

x_valid = valid_df[independent_cols].copy()
y_valid = valid_df[target_col].copy()

x_test = test_df[independent_cols].copy()
y_test = test_df[target_col].copy()

## Artificial Neural Network Implementation

In [16]:
def build_model(hp):
    model = keras.Sequential()
    # no. of hidden layers should be present b/w 2 to 20
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(keras.layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',
        metrics=['accuracy'])
    return model

In [17]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 98 kB 3.4 MB/s 


In [20]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='/gdrive/MyDrive/Covid Prediction AI Project',
    project_name='covitest')

INFO:tensorflow:Reloading Oracle from existing project /gdrive/MyDrive/Covid Prediction AI Project/covitest/oracle.json
INFO:tensorflow:Reloading Tuner from /gdrive/MyDrive/Covid Prediction AI Project/covitest/tuner0.json


In [21]:
tuner.search_space_summary()

Search space summary
Default search space size: 14
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_3 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_4 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_5 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_6 (Int)
{'default': None, 'co

In [22]:
tuner.search(x_train, y_train,
             epochs=3,
             validation_data=(x_valid, y_valid))

Trial 6 Complete [00h 11m 28s]
val_accuracy: 0.851714034875234

Best val_accuracy So Far: 0.852867841720581
Total elapsed time: 00h 31m 06s
INFO:tensorflow:Oracle triggered exit


In [23]:
tuner.results_summary()

Results summary
Results in /gdrive/MyDrive/Covid Prediction AI Project/covitest
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
num_layers: 3
units_0: 256
units_1: 384
learning_rate: 0.0001
units_2: 192
units_3: 32
units_4: 192
units_5: 128
units_6: 480
units_7: 128
units_8: 320
units_9: 512
units_10: 160
units_11: 384
Score: 0.852867841720581
Trial summary
Hyperparameters:
num_layers: 8
units_0: 64
units_1: 32
learning_rate: 0.01
units_2: 448
units_3: 96
units_4: 224
units_5: 416
units_6: 32
units_7: 352
units_8: 448
units_9: 320
units_10: 448
units_11: 64
Score: 0.8523101806640625
Trial summary
Hyperparameters:
num_layers: 6
units_0: 480
units_1: 448
learning_rate: 0.01
units_2: 224
units_3: 448
units_4: 192
units_5: 160
units_6: 448
units_7: 160
units_8: 384
units_9: 448
units_10: 64
units_11: 96
Score: 0.8521755735079447
Trial summary
Hyperparameters:
num_layers: 12
units_0: 320
units_1: 224
learning_rate: 0.01
units_2: 32
units

In [28]:
model=tuner.get_best_models()[0]

In [29]:
model.summary()

ValueError: ignored

In [ ]:
model.fit(x_train, y_train,
             epochs=20,
             validation_data=(x_valid, y_valid), initial_epoch=3)

Epoch 4/100
11701/11701 [==============================] - 57s 5ms/step - loss: 0.3550 - accuracy: 0.8527 - val_loss: 0.3558 - val_accuracy: 0.8527
Epoch 5/100
11701/11701 [==============================] - 56s 5ms/step - loss: 0.3545 - accuracy: 0.8529 - val_loss: 0.3558 - val_accuracy: 0.8518
Epoch 6/100
11701/11701 [==============================] - 53s 4ms/step - loss: 0.3543 - accuracy: 0.8530 - val_loss: 0.3558 - val_accuracy: 0.8530
Epoch 7/100
11701/11701 [==============================] - 52s 4ms/step - loss: 0.3541 - accuracy: 0.8533 - val_loss: 0.3555 - val_accuracy: 0.8529
Epoch 8/100
11701/11701 [==============================] - 52s 4ms/step - loss: 0.3540 - accuracy: 0.8533 - val_loss: 0.3557 - val_accuracy: 0.8530
Epoch 9/100
11701/11701 [==============================] - 52s 4ms/step - loss: 0.3539 - accuracy: 0.8534 - val_loss: 0.3554 - val_accuracy: 0.8530
Epoch 10/100
11701/11701 [==============================] - 52s 4ms/step - loss: 0.3538 - accuracy: 0.8537 - val